In [1]:
import tensorflow as tf

/home/krohak/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from utils import *

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [4]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

In [5]:
type(features)

scipy.sparse.lil.lil_matrix

In [6]:
#print(adj[0]
import numpy as np
from collections import Counter
#y_train.tolist()


## Creating the weight matrix

In [ ]:
import networkx as nx
G = nx.Graph(adj)
#G.add_path([0,1,2])
#list(nx.bfs_edges([0,1,2],0))

In [ ]:
paths = nx.all_simple_paths(G, source=0, target=1378,cutoff=2)

In [ ]:
paths=[p for p in paths]
len(paths)

In [ ]:
def memoize(f):
    memo={}

    def helper(g,x,y,z):
        if (x,y,z) not in memo:
            if (y,x,z) not in memo:
                memo[(x,y,z)]=f(g,i=x,j=y,cutoff=z)
                #print(memo)
                return memo[(x,y,z)]
            
            return memo[(y,x,z)]
        return memo[(x,y,z)]

    return helper

In [ ]:
@memoize
def calculate_path_weight(G, i, j, cutoff):
    """https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.simple_paths.all_simple_paths.html"""
    # put G = nx.Graph(adj) before calling the function
    paths = nx.all_simple_paths(G, source=i, target=j,cutoff=cutoff)
    paths=[p for p in paths]
    return len(paths)

In [ ]:
path_len = calculate_path_weight(G,0,1378,2)

In [ ]:
path_len

In [ ]:
features.shape

## Lets create Weight matrix (k=1 and k=2) for i=0

In [ ]:
w_k1 = []
w_k2 = []
for j in range(features.shape[0]):
    w_k1.append(calculate_path_weight(G,0,j,1))
    w_k2.append(calculate_path_weight(G,0,j,2))

In [ ]:
import numpy as np

In [ ]:
w_k1=np.asarray(w_k1,np.float32)
w_k2=np.asarray(w_k2,np.float32)

In [ ]:
w_k1 = w_k1.reshape([w_k1.shape[0],1])
w_k2 = w_k2.reshape([w_k2.shape[0],1])

In [ ]:
w_k1.shape

In [ ]:
w_k2.shape

In [ ]:
(w_k1*w_k2).sum()

In [ ]:
tensor = tf.matmul(w_k1.T,w_k2)

In [ ]:
with tf.Session() as sess:
    print(tensor.eval())

## Lets multiply it by the feature vector, Xc for c = 1

In [ ]:
#features = preprocess_features(features)

In [ ]:
f=features.todense()[0:,1]

In [ ]:
s = w_k1.T*f

In [ ]:
s

.



.

# SECTION 2

.
multi-filter:

https://github.com/krohak/CADL/blob/master/session-3/lecture-3.ipynb

https://github.com/krohak/CADL/blob/master/session-3/session-3.ipynb

https://github.com/krohak/CADL/blob/master/session-3/libs/utils.py

optimizer:

https://github.com/krohak/CADL/blob/master/session-2/lecture-2.ipynb

https://github.com/krohak/CADL/blob/master/session-2/session-2.ipynb
.

In [ ]:
var_gs = {}
name = '01'
Fl = 8
Kl = 2
Cl = features.shape[1]
Nl = features.shape[0]

In [ ]:
from inits import *

In [ ]:
with tf.variable_scope( name + '_vars'):
        for k in range(Kl):
                for c in range(Cl):
                    var_gs['weights_' + str(k+1) + '_' + str(c+1)] = glorot([1,Fl],
                                                            name=('weights_' + str(k+1) + '_' + str(c+1)))

In [ ]:
var_gs

To get a list of current nodes in graph

In [ ]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

Clear the graph

In [ ]:
var_gs = {}
name = '01'
Fl = 8
Kl = 2
Cl = features.shape[1]
Nl = features.shape[0]
from tensorflow.python.framework.ops import reset_default_graph
reset_default_graph()

In [37]:
g = tf.get_default_graph()
[op.name for op in g.get_operations()]

['Placeholder',
 'Placeholder_1',
 '01_vars/strided_slice/stack',
 '01_vars/strided_slice/stack_1',
 '01_vars/strided_slice/stack_2',
 '01_vars/strided_slice',
 '01_vars/MatMul',
 '01_vars/G_k_0/Initializer/random_uniform/shape',
 '01_vars/G_k_0/Initializer/random_uniform/min',
 '01_vars/G_k_0/Initializer/random_uniform/max',
 '01_vars/G_k_0/Initializer/random_uniform/RandomUniform',
 '01_vars/G_k_0/Initializer/random_uniform/sub',
 '01_vars/G_k_0/Initializer/random_uniform/mul',
 '01_vars/G_k_0/Initializer/random_uniform',
 '01_vars/G_k_0',
 '01_vars/G_k_0/Assign',
 '01_vars/G_k_0/read',
 '01_vars/MatMul_1',
 '01_vars/Add/x',
 '01_vars/Add',
 '01_vars/strided_slice_1/stack',
 '01_vars/strided_slice_1/stack_1',
 '01_vars/strided_slice_1/stack_2',
 '01_vars/strided_slice_1',
 '01_vars/MatMul_2',
 '01_vars/G_k_1/Initializer/random_uniform/shape',
 '01_vars/G_k_1/Initializer/random_uniform/min',
 '01_vars/G_k_1/Initializer/random_uniform/max',
 '01_vars/G_k_1/Initializer/random_uniform/Ra

In [ ]:
#s = tf.constant(s[0,0])

In [ ]:
t = s[0,0]*var_gs['weights_' + str(1) + '_' + str(1)]
z = s[0,0]*var_gs['weights_' + str(1) + '_' + str(1)]
#with sess.as_default():
#t.eval()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run((t,z,t+z))
type(t.eval(session=sess))

In [ ]:
def dot(x, y, sparse=False):
    """Wrapper for tf.matmul (sparse vs dense)."""
    if sparse:
        res = tf.sparse_tensor_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res

In [ ]:
dot(s[0,0], var_gs['weights_' + str(1) + '_' + str(1)])

.



.

In [ ]:
import networkx as nx
G = nx.Graph(adj)

In [ ]:
def memoize(f):
    memo={}

    def helper(g,x,y,z):
        if (x,y,z) not in memo:
            if (y,x,z) not in memo:
                p=f(g,i=x,j=y,cutoff=z)
                memo[(x,y,z)]=p
                print(x,y,p)
                return memo[(x,y,z)]
            
            return memo[(y,x,z)]
        return memo[(x,y,z)]

    return helper

In [ ]:
@memoize
def calculate_path_weight(G, i, j, cutoff):
    """https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.simple_paths.all_simple_paths.html"""
    # put G = nx.Graph(adj) before calling the function
    paths = nx.all_simple_paths(G, source=i, target=j,cutoff=cutoff)
    paths=[p for p in paths]
    return len(paths)

In [ ]:
for i in range(features.shape[0]):
    for k in range(1,Kl):
        for j in range(features.shape[0]):
            calculate_path_weight(G,i,j,k)

.



.

In [8]:
a = np.array([1, 2, 3])
b = np.array([2, 3, 4])
a = np.vstack((a,b))

In [11]:
a = np.vstack((a,a))

In [12]:
a

array([[1, 2, 3],
       [2, 3, 4],
       [1, 2, 3],
       [2, 3, 4]])

# SECTION 3

In [1]:
import tensorflow as tf
import numpy as np

/home/krohak/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [104]:
from utils import *
from metrics import *

In [3]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [157]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)
#features = preprocess_features(features)

In [158]:
features = features.todense()

In [159]:
path_weight_matrix = np.load("path_weights.dat")
path_weight_matrix.shape
path_weight_matrix = path_weight_matrix.astype('float32')

In [160]:
path_weight_matrix[1,:,1].shape

(2708,)

In [161]:
type(features)

numpy.matrixlib.defmatrix.matrix

In [162]:
f=features
output=[]
output = np.asarray(output,dtype=np.float32)
path_weight_matrix.dtype, path_weight_matrix.shape

(dtype('float32'), (2708, 2708, 2))

In [163]:
features.shape

(2708, 1433)

In [220]:
g = tf.get_default_graph()
[op.name for op in g.get_operations()]

[]

In [228]:
from inits import *
var_gs = {}
name = '01'
Fl = 8
Kl = 2
Cl = features.shape[1]
Nl = features.shape[0]
from tensorflow.python.framework.ops import reset_default_graph
reset_default_graph()

In [229]:
# features_m = tf.sparse_placeholder(tf.float32, shape=[features[2][0], features[2][1]])
features_m = tf.placeholder(tf.float32, shape=[features.shape[0], features.shape[1]])
pwm = tf.placeholder(tf.float32, [Nl, Nl, Kl])
outputs=[]

conv = np.zeros([Nl,Fl],dtype=np.float32)
#tf.placeholder(tf.float32, shape=[Nl,Fl])

# droupout
features_m = tf.nn.dropout(features_m, 1-FLAGS.dropout)

with tf.variable_scope( name + '_vars'):    
     
    for k in range(Kl):
        
        w_k = pwm[:,:,k]
        
#         s = tf.sparse_tensor_dense_matmul(sp_a=features_m, b=w_k)
        s = tf.matmul(w_k,features_m)
    
        G_k = tf.get_variable(name=('G_k_%s'%(k)),initializer=tf.contrib.layers.xavier_initializer(),shape=[Cl,Fl])
        
        res = tf.matmul(s,G_k)
        
        outputs.append(res)
        
        conv = tf.add(conv,res)
        
        print(conv.shape)
    
        outputs.append(conv)
    
# add bias
initial = tf.zeros([Nl,Fl], dtype=tf.float32)
bias = tf.Variable(initial, name='bias')

conv = tf.add(conv,bias)

# apply non-linearity
conv = tf.nn.relu(conv)

(2708, 8)
(2708, 8)


In [230]:
with tf.variable_scope( name + '_vars'):    
     
    for k in range(Kl):
        
        w_k = pwm[:,:,k]
        
        s = tf.matmul(w_k,conv)
    
        G_k = tf.get_variable(name=('G_k2_%s'%(k)),initializer=tf.contrib.layers.xavier_initializer(),shape=[Fl,Fl])
        
        res = tf.matmul(s,G_k)
        
        outputs.append(res)
        
        conv = tf.add(conv,res)
        
        print(conv.shape)
    
        outputs.append(conv)

# add bias
initial = tf.zeros([Nl,Fl], dtype=tf.float32)
bias = tf.Variable(initial, name='bias2')

conv = tf.add(conv,bias)

# apply non-linearity
conv = tf.nn.relu(conv)

(2708, 8)
(2708, 8)


In [231]:
with tf.variable_scope( name + '_vars'):    
             
    w_k = pwm[:,:,k]

    s = tf.matmul(w_k,conv)

    G_k = tf.get_variable(name=('G_k3_%s'%(k)),initializer=tf.contrib.layers.xavier_initializer(),shape=[Fl,7])

    conv = tf.matmul(s,G_k)
    

print(conv.shape)    
# add bias
initial = tf.zeros([Nl,7], dtype=tf.float32)
bias = tf.Variable(initial, name='bias3')
conv = tf.add(conv,bias)


# apply non-linearity
conv = tf.nn.relu(conv)

(2708, 7)


In [232]:
outputs

[<tf.Tensor '01_vars/MatMul_1:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars/Add:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars/MatMul_3:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars/Add_1:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars_1/MatMul_1:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars_1/Add:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars_1/MatMul_3:0' shape=(2708, 8) dtype=float32>,
 <tf.Tensor '01_vars_1/Add_1:0' shape=(2708, 8) dtype=float32>]

In [233]:
#conv = tf.nn.softmax(conv)

accuracy = masked_accuracy(conv,y_train, train_mask)


In [234]:
# one hot encoded
y_train.shape

(2708, 7)

In [235]:
loss= 0

optimizer = tf.train.AdamOptimizer(learning_rate=FLAGS.learning_rate)

loss += masked_softmax_cross_entropy(conv, y_train, train_mask)

opt_op = optimizer.minimize(loss)

In [236]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
evals = sess.run([opt_op,accuracy,loss],feed_dict={features_m:features,pwm:path_weight_matrix})
#type(t.eval(session=sess))

In [237]:
evals[1], evals[2]

(0.13571428, 306173.09)

In [238]:
for epoch in range(FLAGS.epochs):
    
    evals = sess.run([opt_op,accuracy,loss],feed_dict={features_m:features,pwm:path_weight_matrix})
    print(evals[1], evals[2])

0.142857 466753.0
0.157143 19003.6
0.385714 1.90367
0.485714 1.68565
0.542857 1.6353
0.578572 1.58931
0.578572 1.54648
0.585714 1.50637
0.585714 1.46876
0.585714 1.43336
0.585714 1.39998
0.585714 1.36849
0.592857 1.33879
0.592857 1.31074
0.607143 1.28433
0.607143 1.25971
0.621429 1.23607
0.642857 1.21354
0.657143 1.19216
0.7 1.17132
0.7 1.15184
0.7 1.13484
0.707143 1.11924
0.728572 1.10482
0.721429 1.09133
0.742857 1.07858
0.75 1.06626
0.757143 1.05433
0.75 1.04281
0.757143 1.03166
0.771429 1.0208
0.785714 1.01027
0.792857 1.00004
0.821429 0.990092
0.828572 0.980478
0.835714 0.971162
0.842857 0.962064
0.842857 0.953185
0.842857 0.944522
0.857143 0.936096
0.857143 0.927869
0.857143 0.919905
0.857143 0.912214
0.857143 0.90472
0.857143 0.897392
0.864286 0.890223
0.864286 0.883209
0.864286 0.876351
0.864286 0.869648
0.864286 0.863101
0.864286 0.856724
0.857143 0.850521
0.857143 0.8445
0.857143 0.838622
0.85 0.832888


KeyboardInterrupt: 

Reusing variables:
https://www.tensorflow.org/versions/r0.12/how_tos/variable_scope/

In [ ]:
f=features.todense()
output=[]
output = np.asarray(output,dtype=np.float32)
with tf.variable_scope( name + '_vars') as scope:
    scope.reuse_variables()
    for i in range(features.shape[0]):
        conv = tf.get_variable("conv",shape=())
        for k in range(Kl):
            for c in range(Cl):
                var_gs['weights_' + str(k+1) + '_' + str(c+1)] = glorot([1,Fl],name=('weights_' + str(k+1) + '_' + str(c+1)))
                w_k = []
                w_k = path_weight_matrix[i,:,k]
                w_k = np.asarray(w_k,np.float32)
                w_k = w_k.reshape([w_k.shape[0],1])
                x_c = f[0:,c]
                s = tf.matmul(w_k.T,x_c)
                conv = tf.add(conv,tf.multiply(s[0,0],var_gs['weights_' + str(k+1) + '_' + str(c+1)]))  
        print(conv)  

# SECTION 4

.

Installing TF with GPU:
https://www.tensorflow.org/install/install_linux

Check available GPUs on TF:
https://stackoverflow.com/questions/38559755/how-to-get-current-available-gpus-in-tensorflow

Change port of SSH:
https://wiki.archlinux.org/index.php/Secure_Shell

Check GPU info:
https://askubuntu.com/questions/5417/how-to-get-the-gpu-info

Distributed TF:
https://www.tensorflow.org/deploy/distributed

.

Xavier (Glorot) initializer: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/xavier_initializer

Mutmul exceptions: https://stackoverflow.com/questions/34908033/tensorflow-exception-with-matmul

tf.Variable: https://www.tensorflow.org/api_docs/python/tf/Variable



.



.

In [ ]:
print(list(nx.bfs_tree(G,0,0)))

In [ ]:
print(nx.shortest_path_length(G,source=0))

In [ ]:
one_path = nx.all_pairs_shortest_path(G,cutoff=1)[0]
two_path = nx.all_pairs_shortest_path(G,cutoff=2)[0]

In [ ]:
print(len(two_path), len(one_path)-1, len(two_path)-len(one_path)-1)

In [ ]:
one_path

In [ ]:
two_path

In [ ]:
type(two_path)

In [ ]:
features = preprocess_features(features)

In [ ]:
#print(features.shape)
features[1].shape
#2708*1433

In [ ]:
print(adj.toarray())

In [ ]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from utils import *
from models import GCN, MLP

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('dataset', 'pubmed', 'Dataset string.')  # 'cora', 'citeseer', 'pubmed'
flags.DEFINE_string('model', 'gcn', 'Model string.')  # 'gcn', 'gcn_cheby', 'dense'
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

# Load data
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(FLAGS.dataset)

# Some preprocessing
features = preprocess_features(features)
if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
elif FLAGS.model == 'gcn_cheby':
    support = chebyshev_polynomials(adj, FLAGS.max_degree)
    num_supports = 1 + FLAGS.max_degree
    model_func = GCN
elif FLAGS.model == 'dense':
    support = [preprocess_adj(adj)]  # Not used
    num_supports = 1
    model_func = MLP
else:
    raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
sess = tf.Session()


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Construct feed dictionary
feed_dict = construct_feed_dict(features, support, y_train, train_mask, placeholders)
feed_dict.update({placeholders['dropout']: FLAGS.dropout})


# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
  

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, test_duration = evaluate(features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))


In [ ]:
y_val.shape

In [ ]:
features